In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

In [6]:
df = pd.read_csv("../data/MiniP2/2022_tmoney.csv")
df.head()

,사용월,노선ID,노선번호,노선명,버스정류장ID,버스정류장명,승차승객수,하차승객수,작업일시
0,2022-01,11110897,0017,0017번(청암동~이촌동),1,청암자이아파트,453,12,2022-02-03 08:49:57
1,2022-01,11110897,0017,0017번(청암동~이촌동),2,청암동강변삼성아파트,"1,328",43,2022-02-03 08:49:57
2,2022-01,11110897,0017,0017번(청암동~이촌동),3,청심경로당,"2,420",45,2022-02-03 08:49:57
3,2022-01,11110897,0017,0017번(청암동~이촌동),4,원효2동주민센터,"4,196",95,2022-02-03 08:49:57
4,2022-01,11110897,0017,0017번(청암동~이촌동),5,산천동,"5,255",206,2022-02-03 08:49:57


In [7]:
# 필요없는 데이터 제거
df = df.drop("작업일시" ,axis=1)

In [8]:
df.shape

(358608, 8)

In [9]:
# 승객수 연산을 위해 타입 변경
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358608 entries, 0 to 358607
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   사용월      358608 non-null  object
 1   노선ID     358608 non-null  int64 
 2   노선번호     358608 non-null  object
 3   노선명      358608 non-null  object
 4   버스정류장ID  358608 non-null  int64 
 5   버스정류장명   358608 non-null  object
 6   승차승객수    358608 non-null  object
 7   하차승객수    358608 non-null  object
dtypes: int64(2), object(6)
memory usage: 21.9+ MB


In [10]:
# 문자 타입 숫자 단위 구분자 삭제
df["승차승객수"] = df["승차승객수"].str.replace(",", "")
df["하차승객수"] = df["하차승객수"].str.replace(",", "")

In [11]:
# int 로 변환
df["승차승객수"] = pd.to_numeric(df["승차승객수"])
df["하차승객수"] = pd.to_numeric(df["하차승객수"])

In [12]:
# 승객수 타입 변경 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358608 entries, 0 to 358607
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   사용월      358608 non-null  object
 1   노선ID     358608 non-null  int64 
 2   노선번호     358608 non-null  object
 3   노선명      358608 non-null  object
 4   버스정류장ID  358608 non-null  int64 
 5   버스정류장명   358608 non-null  object
 6   승차승객수    358608 non-null  int64 
 7   하차승객수    358608 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 21.9+ MB


In [13]:
# 승객수 연산
df["이용객"] = df["승차승객수"] + df["하차승객수"]
df

,사용월,노선ID,노선번호,노선명,버스정류장ID,버스정류장명,승차승객수,하차승객수,이용객
0,2022-01,11110897,0017,0017번(청암동~이촌동),1,청암자이아파트,453,12,465
1,2022-01,11110897,0017,0017번(청암동~이촌동),2,청암동강변삼성아파트,1328,43,1371
2,2022-01,11110897,0017,0017번(청암동~이촌동),3,청심경로당,2420,45,2465
3,2022-01,11110897,0017,0017번(청암동~이촌동),4,원효2동주민센터,4196,95,4291
4,2022-01,11110897,0017,0017번(청암동~이촌동),5,산천동,5255,206,5461
...,...,...,...,...,...,...,...,...,...
358603,2022-09,11110729,중랑02,중랑02(진로아파트~한신아파트),22,서일대정문,141,6056,6197
358604,2022-09,11110729,중랑02,중랑02(진로아파트~한신아파트),23,새마을금고,586,5616,6202
358605,2022-09,11110729,중랑02,중랑02(진로아파트~한신아파트),24,공판장,43,7038,7081
358606,2022-09,11110729,중랑02,중랑02(진로아파트~한신아파트),25,진주빌라.바다약국,29,9723,9752


* 노선별, 버스정류장 id별 그룹화
* 정류장 id 별 위경도
* 승하차가 많은 곳은 정류장 차선 분리
* 정류장명 별 그룹화 -> 한 정류장을 지나는 여러 노선 존재 -> 그 정류장에서 승차하는 승객 노선별 비율 

In [14]:
# 정류장 별 이용객 합계 그룹화
df_temp = df.groupby(["사용월", "버스정류장명"])[["승차승객수", "하차승객수", "이용객"]].sum()
df_temp = df_temp.rename(columns={"승차승객수":"승차승객수 합", "하차승객수":'하차승객수 합', "이용객":"이용객 합"})
df_temp

승차승객수 합  하차승객수 합  이용객 합
사용월     버스정류장명                                  
2022-01 (구)경남호텔앞            3556    17016  20572
        (구)법화사               454      697   1151
        1156번종점             1166     1104   2270
        11단지                5341     1038   6379
        123전자타운.2001아울렛    13626    16365  29991
...                          ...      ...    ...
2022-09 힐스테이트판교엘포레4단지        830      199   1029
        힐스테이트판교엘포레6단지        188      212    400
        힐스테이트판교엘포레6단지후문      237       91    328
        힐탑                   111      668    779
        힐튼호텔               10501    30007  40508

[72251 rows x 3 columns]

In [90]:
# 정류장, 노선별 이용객 그룹화
df_temp2 = df.groupby(["사용월", "버스정류장명", "버스정류장ID", "노선명"])[["승차승객수", "하차승객수", "이용객"]].sum()
df_temp2

승차승객수  하차승객수    이용객
사용월     버스정류장명   버스정류장ID 노선명                                     
2022-01 (구)경남호텔앞 28      2233번(면목동~옥수동)        1380   4304   5684
                 68      2416번(중랑공영차고지~삼성역)     781   4292   5073
                 94      2112번(면목동~성북동)         614   4105   4719
                 107     2233번(면목동~옥수동)         781   4315   5096
        (구)법화사   3       성북02(한성대학교~우리옛돌박물관)    443     13    456
...                                             ...    ...    ...
2022-09 힐튼호텔     12      01번(예장주차장~예장주차장)      1367   9979  11346
                 42      405번(염곡동~시청광장)         533   3983   4516
                 50      405번(염곡동~시청광장)        2198   1664   3862
                 52      402번(장지공영차고지~광화문)     1006   9819  10825
                 61      402번(장지공영차고지~광화문)     5397   4562   9959

[358608 rows x 3 columns]

In [94]:
# 위 두 그룹 병합
df_merge = pd.merge(df_temp.reset_index(), df_temp2.reset_index(), on=["사용월", "버스정류장명"])

In [95]:
df_merge.head(10)

,사용월,버스정류장명,승차승객수 합,하차승객수 합,이용객 합,버스정류장ID,노선명,승차승객수,하차승객수,이용객
0,2022-01,(구)경남호텔앞,3556,17016,20572,28,2233번(면목동~옥수동),1380,4304,5684
1,2022-01,(구)경남호텔앞,3556,17016,20572,68,2416번(중랑공영차고지~삼성역),781,4292,5073
2,2022-01,(구)경남호텔앞,3556,17016,20572,94,2112번(면목동~성북동),614,4105,4719
3,2022-01,(구)경남호텔앞,3556,17016,20572,107,2233번(면목동~옥수동),781,4315,5096
4,2022-01,(구)법화사,454,697,1151,3,성북02(한성대학교~우리옛돌박물관),443,13,456
5,2022-01,(구)법화사,454,697,1151,36,성북02(한성대학교~우리옛돌박물관),11,684,695
6,2022-01,1156번종점,1166,1104,2270,1,1156번(퇴계원쌍용예가아파트~석계역),1164,17,1181
7,2022-01,1156번종점,1166,1104,2270,73,1156번(퇴계원쌍용예가아파트~석계역),2,1087,1089
8,2022-01,11단지,5341,1038,6379,10,5530번(군포공영차고지~사당역),5341,1038,6379
9,2022-01,123전자타운.2001아울렛,13626,16365,29991,2,구로06(개봉역~센츄리아파트),3048,6381,9429


In [96]:
# 번호 추출
df_merge["노선번호"] = df_merge["노선명"].copy()

In [97]:
df_merge["노선번호"] = df_merge["노선번호"].map(lambda x : x.split('(')[0])

In [98]:
df_merge["노선번호"] = df_merge["노선번호"].str.replace("번", "")

In [99]:
# 컬럼 재배치
df_merge = df_merge[["사용월", "노선번호", "노선명", "버스정류장명", "버스정류장ID", "승차승객수", "하차승객수", "이용객", "승차승객수 합", "하차승객수 합", "이용객 합"]]
df_merge.head(10)

,사용월,노선번호,노선명,버스정류장명,버스정류장ID,승차승객수,하차승객수,이용객,승차승객수 합,하차승객수 합,이용객 합
0,2022-01,2233,2233번(면목동~옥수동),(구)경남호텔앞,28,1380,4304,5684,3556,17016,20572
1,2022-01,2416,2416번(중랑공영차고지~삼성역),(구)경남호텔앞,68,781,4292,5073,3556,17016,20572
2,2022-01,2112,2112번(면목동~성북동),(구)경남호텔앞,94,614,4105,4719,3556,17016,20572
3,2022-01,2233,2233번(면목동~옥수동),(구)경남호텔앞,107,781,4315,5096,3556,17016,20572
4,2022-01,성북02,성북02(한성대학교~우리옛돌박물관),(구)법화사,3,443,13,456,454,697,1151
5,2022-01,성북02,성북02(한성대학교~우리옛돌박물관),(구)법화사,36,11,684,695,454,697,1151
6,2022-01,1156,1156번(퇴계원쌍용예가아파트~석계역),1156번종점,1,1164,17,1181,1166,1104,2270
7,2022-01,1156,1156번(퇴계원쌍용예가아파트~석계역),1156번종점,73,2,1087,1089,1166,1104,2270
8,2022-01,5530,5530번(군포공영차고지~사당역),11단지,10,5341,1038,6379,5341,1038,6379
9,2022-01,구로06,구로06(개봉역~센츄리아파트),123전자타운.2001아울렛,2,3048,6381,9429,13626,16365,29991


In [100]:
df_merge.sort_values(["버스정류장명", "사용월"]).head(10)

,사용월,노선번호,노선명,버스정류장명,버스정류장ID,승차승객수,하차승객수,이용객,승차승객수 합,하차승객수 합,이용객 합
0,2022-01,2233,2233번(면목동~옥수동),(구)경남호텔앞,28,1380,4304,5684,3556,17016,20572
1,2022-01,2416,2416번(중랑공영차고지~삼성역),(구)경남호텔앞,68,781,4292,5073,3556,17016,20572
2,2022-01,2112,2112번(면목동~성북동),(구)경남호텔앞,94,614,4105,4719,3556,17016,20572
3,2022-01,2233,2233번(면목동~옥수동),(구)경남호텔앞,107,781,4315,5096,3556,17016,20572
39159,2022-02,2233,2233번(면목동~옥수동),(구)경남호텔앞,28,1029,3360,4389,2951,14451,17402
39160,2022-02,2416,2416번(중랑공영차고지~삼성역),(구)경남호텔앞,68,717,4229,4946,2951,14451,17402
39161,2022-02,2112,2112번(면목동~성북동),(구)경남호텔앞,94,622,3333,3955,2951,14451,17402
39162,2022-02,2233,2233번(면목동~옥수동),(구)경남호텔앞,107,583,3529,4112,2951,14451,17402
78241,2022-03,2233,2233번(면목동~옥수동),(구)경남호텔앞,28,1211,3744,4955,3572,16143,19715
78242,2022-03,2416,2416번(중랑공영차고지~삼성역),(구)경남호텔앞,68,969,4973,5942,3572,16143,19715


In [101]:
# 특정 정류장에서 어느 노선 이용 비율이 더 높은가
df_merge["이용객 pct"] = round(df_merge["이용객"] / df_merge["이용객 합"] * 100, 2)

In [102]:
df_merge

,사용월,노선번호,노선명,버스정류장명,버스정류장ID,승차승객수,하차승객수,이용객,승차승객수 합,하차승객수 합,이용객 합,이용객 pct
0,2022-01,2233,2233번(면목동~옥수동),(구)경남호텔앞,28,1380,4304,5684,3556,17016,20572,27.63
1,2022-01,2416,2416번(중랑공영차고지~삼성역),(구)경남호텔앞,68,781,4292,5073,3556,17016,20572,24.66
2,2022-01,2112,2112번(면목동~성북동),(구)경남호텔앞,94,614,4105,4719,3556,17016,20572,22.94
3,2022-01,2233,2233번(면목동~옥수동),(구)경남호텔앞,107,781,4315,5096,3556,17016,20572,24.77
4,2022-01,성북02,성북02(한성대학교~우리옛돌박물관),(구)법화사,3,443,13,456,454,697,1151,39.62
...,...,...,...,...,...,...,...,...,...,...,...,...
358603,2022-09,01,01번(예장주차장~예장주차장),힐튼호텔,12,1367,9979,11346,10501,30007,40508,28.01
358604,2022-09,405,405번(염곡동~시청광장),힐튼호텔,42,533,3983,4516,10501,30007,40508,11.15
358605,2022-09,405,405번(염곡동~시청광장),힐튼호텔,50,2198,1664,3862,10501,30007,40508,9.53
358606,2022-09,402,402번(장지공영차고지~광화문),힐튼호텔,52,1006,9819,10825,10501,30007,40508,26.72


In [109]:
df_merge.sort_values(["이용객 pct"]).head(10)

,사용월,노선번호,노선명,버스정류장명,버스정류장ID,승차승객수,하차승객수,이용객,승차승객수 합,하차승객수 합,이용객 합,이용객 pct
36178,2022-01,N852,N852번(신림동~건국대),프리마호텔,43,1,1,2,128305,118259,246564,0.0
182395,2022-05,362,362번(송파공영차고지~여의도),신사중학교,36,3,2,5,121178,106603,227781,0.0
196542,2022-05,362,362번(송파공영차고지~여의도),현대아파트,70,1,2,3,276697,254375,531072,0.0
182403,2022-05,362,362번(송파공영차고지~여의도),신사중학교,68,0,1,1,121178,106603,227781,0.0
62534,2022-02,8541,8541번(호압사~강남역),신논현역.구교보타워사거리,39,2,3,5,87200,44617,131817,0.0
221535,2022-06,금천01-1,금천01-1(가산디지털단지역~벽산아파트),시흥동은행나무,16,0,1,1,68728,39493,108221,0.0
161558,2022-05,362,362번(송파공영차고지~여의도),광림교회.현대고등학교,35,1,1,2,79982,65835,145817,0.0
151256,2022-04,N51,N51번(시흥동차고지~하계동차고지),종암사거리입구,100,10,3,13,155398,143480,298878,0.0
189915,2022-05,362,362번(송파공영차고지~여의도),잠실엘스아파트앞,22,0,4,4,46796,49964,96760,0.0
161567,2022-05,362,362번(송파공영차고지~여의도),광림교회.현대고등학교,69,5,0,5,79982,65835,145817,0.0


# 종현님의 위경도랑 합치기

In [104]:
# 종현님의 정류장별 위경도 데이터
df_test = pd.read_csv("../data/MiniP2/2022_tmoney_xy.csv")
df_test.sort_values("정류소명")

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y,사용월,노선ID,노선번호,노선명,버스정류장ID,버스정류장명,승차승객수,하차승객수,작업일시
120218,105000159,6245,(구)경남호텔앞,127.070008,37.569610,2022-09,11110950,2416,2416번(중랑공영차고지~삼성역),68,(구)경남호텔앞,"1,378","6,394",2022-10-03 09:13:34
120200,105000159,6245,(구)경남호텔앞,127.070008,37.569610,2022-05,11110193,2233,2233번(면목동~옥수동),28,(구)경남호텔앞,"1,402","4,362",2022-06-13 17:02:22
120199,105000159,6245,(구)경남호텔앞,127.070008,37.569610,2022-05,11110178,2112,2112번(면목동~성북동),94,(구)경남호텔앞,782,"4,413",2022-06-13 17:02:22
120198,105000159,6245,(구)경남호텔앞,127.070008,37.569610,2022-04,11110950,2416,2416번(중랑공영차고지~삼성역),68,(구)경남호텔앞,970,"5,625",2022-05-03 09:52:32
120197,105000159,6245,(구)경남호텔앞,127.070008,37.569610,2022-04,11110193,2233,2233번(면목동~옥수동),107,(구)경남호텔앞,741,"4,253",2022-05-03 09:52:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45844,101000265,2285,힐튼호텔,126.976612,37.556606,2022-07,11110046,402,402번(장지공영차고지~광화문),52,힐튼호텔,"1,069","9,678",2022-08-03 09:32:37
45845,101000265,2285,힐튼호텔,126.976612,37.556606,2022-07,11110046,402,402번(장지공영차고지~광화문),61,힐튼호텔,"5,356","4,523",2022-08-03 09:32:37
45846,101000265,2285,힐튼호텔,126.976612,37.556606,2022-07,11110337,405,405번(염곡동~시청광장),42,힐튼호텔,559,"4,137",2022-08-03 09:32:37
45835,101000265,2285,힐튼호텔,126.976612,37.556606,2022-05,11110046,402,402번(장지공영차고지~광화문),61,힐튼호텔,"5,924","4,569",2022-06-13 17:02:22


In [114]:
# 저 힐튼호텔 04 뭐야
df_xy = df_test[["노선번호", "정류소명", "버스정류장ID", "좌표X", "좌표Y"]]
df_xy = df_xy.drop_duplicates()
df_xy = df_xy.sort_values("정류소명")
df_xy

,노선번호,정류소명,버스정류장ID,좌표X,좌표Y
120183,2112,(구)경남호텔앞,94,127.070008,37.569610
120184,2233,(구)경남호텔앞,28,127.070008,37.569610
120185,2233,(구)경남호텔앞,107,127.070008,37.569610
120186,2416,(구)경남호텔앞,68,127.070008,37.569610
179904,성북02,(구)법화사,36,127.011093,37.584588
...,...,...,...,...,...
45766,04,힐튼호텔,4,126.976653,37.556204
45862,405,힐튼호텔,50,126.977274,37.554832
45878,01,힐튼호텔,12,126.977274,37.554832
45812,04,힐튼호텔,4,126.976612,37.556606


In [120]:
df_test[(df_test["노선번호"] == "04") & (df_test["정류소명"] == "힐튼호텔")]

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y,사용월,노선ID,노선번호,노선명,버스정류장ID,버스정류장명,승차승객수,하차승객수,작업일시
45766,101000266,2284,힐튼호텔,126.976653,37.556204,2022-01,11110446,04,04번(예장주차장~예장주차장),4,힐튼호텔,992,163,2022-02-03 08:49:56
45771,101000266,2284,힐튼호텔,126.976653,37.556204,2022-02,11110446,04,04번(예장주차장~예장주차장),4,힐튼호텔,728,150,2022-03-03 11:02:09
45776,101000266,2284,힐튼호텔,126.976653,37.556204,2022-03,11110446,04,04번(예장주차장~예장주차장),4,힐튼호텔,894,192,2022-04-03 09:19:55
45781,101000266,2284,힐튼호텔,126.976653,37.556204,2022-04,11110446,04,04번(예장주차장~예장주차장),4,힐튼호텔,"1,074",345,2022-05-03 09:52:32
45787,101000266,2284,힐튼호텔,126.976653,37.556204,2022-05,11110446,04,04번(예장주차장~예장주차장),4,힐튼호텔,40,17,2022-06-13 17:02:22
45812,101000265,2285,힐튼호텔,126.976612,37.556606,2022-01,11110446,04,04번(예장주차장~예장주차장),4,힐튼호텔,992,163,2022-02-03 08:49:56
45817,101000265,2285,힐튼호텔,126.976612,37.556606,2022-02,11110446,04,04번(예장주차장~예장주차장),4,힐튼호텔,728,150,2022-03-03 11:02:09
45822,101000265,2285,힐튼호텔,126.976612,37.556606,2022-03,11110446,04,04번(예장주차장~예장주차장),4,힐튼호텔,894,192,2022-04-03 09:19:55
45827,101000265,2285,힐튼호텔,126.976612,37.556606,2022-04,11110446,04,04번(예장주차장~예장주차장),4,힐튼호텔,"1,074",345,2022-05-03 09:52:32
45833,101000265,2285,힐튼호텔,126.976612,37.556606,2022-05,11110446,04,04번(예장주차장~예장주차장),4,힐튼호텔,40,17,2022-06-13 17:02:22


In [119]:
df_merge[(df_merge["노선번호"] == "04") & (df_merge["버스정류장명"] == "힐튼호텔")]

,사용월,노선번호,노선명,버스정류장명,버스정류장ID,승차승객수,하차승객수,이용객,승차승객수 합,하차승객수 합,이용객 합,이용객 pct
39154,2022-01,04,04번(예장주차장~예장주차장),힐튼호텔,4,992,163,1155,7849,15358,23207,4.98
78236,2022-02,04,04번(예장주차장~예장주차장),힐튼호텔,4,728,150,878,6803,12879,19682,4.46
117319,2022-03,04,04번(예장주차장~예장주차장),힐튼호텔,4,894,192,1086,8068,15439,23507,4.62
157529,2022-04,04,04번(예장주차장~예장주차장),힐튼호텔,4,1074,345,1419,10250,20032,30282,4.69
197856,2022-05,04,04번(예장주차장~예장주차장),힐튼호텔,4,40,17,57,10779,28114,38893,0.15


In [112]:
df_xy = df_xy.rename(columns={"정류소명":"버스정류장명"})
df_xy

,버스정류장명,버스정류장ID,좌표X,좌표Y
120185,(구)경남호텔앞,107,127.070008,37.569610
120184,(구)경남호텔앞,28,127.070008,37.569610
120183,(구)경남호텔앞,94,127.070008,37.569610
120186,(구)경남호텔앞,68,127.070008,37.569610
179903,(구)법화사,3,127.011093,37.584588
...,...,...,...,...
45812,힐튼호텔,4,126.976612,37.556606
45815,힐튼호텔,42,126.976612,37.556606
45858,힐튼호텔,4,126.977274,37.554832
45769,힐튼호텔,42,126.976653,37.556204


In [113]:
df_merge.merge(df_xy, how='left', on=["버스정류장명", "버스정류장ID"])

,사용월,노선번호,노선명,버스정류장명,버스정류장ID,승차승객수,하차승객수,이용객,승차승객수 합,하차승객수 합,이용객 합,이용객 pct,좌표X,좌표Y
0,2022-01,2233,2233번(면목동~옥수동),(구)경남호텔앞,28,1380,4304,5684,3556,17016,20572,27.63,127.070008,37.569610
1,2022-01,2416,2416번(중랑공영차고지~삼성역),(구)경남호텔앞,68,781,4292,5073,3556,17016,20572,24.66,127.070008,37.569610
2,2022-01,2112,2112번(면목동~성북동),(구)경남호텔앞,94,614,4105,4719,3556,17016,20572,22.94,127.070008,37.569610
3,2022-01,2233,2233번(면목동~옥수동),(구)경남호텔앞,107,781,4315,5096,3556,17016,20572,24.77,127.070008,37.569610
4,2022-01,성북02,성북02(한성대학교~우리옛돌박물관),(구)법화사,3,443,13,456,454,697,1151,39.62,127.011093,37.584588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718732,2022-09,402,402번(장지공영차고지~광화문),힐튼호텔,52,1006,9819,10825,10501,30007,40508,26.72,126.976653,37.556204
718733,2022-09,402,402번(장지공영차고지~광화문),힐튼호텔,52,1006,9819,10825,10501,30007,40508,26.72,126.977274,37.554832
718734,2022-09,402,402번(장지공영차고지~광화문),힐튼호텔,61,5397,4562,9959,10501,30007,40508,24.59,126.976612,37.556606
718735,2022-09,402,402번(장지공영차고지~광화문),힐튼호텔,61,5397,4562,9959,10501,30007,40508,24.59,126.977274,37.554832
